In [1]:
import requests
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import pandas as pd
import numpy as np

In [2]:
url = "https://fow.kr/neo_ranking.php"

headers = {
    'Cookie': 'FK=FK; _gid=GA1.2.298061248.1700612442; __qca=P0-2093126221-1700612443802; _pbjs_userid_consent_data=3524755945110770; _sharedid=de0d9457-33f9-45b2-8542-0b1802255a79; neo_his=%ED%97%88%EA%B1%B0%EB%8D%A9-0303; _ga=GA1.1.355727062.1700612442; cto_bundle=ACnDo18lMkZScDRXUXFpazNsZWo0MGxQT2NjQWFSeHgyakxYSmVmRHZTSzFHVWtMT1lvV2VseVJrMmFyMHdVVVYlMkJTT2FDZEg1V0FLZCUyRnNTZ0hIMGNoYWNzZFI1YUpKMWVETjlLMDEwUSUyRkRHNkVnWVAzZ3R1M2J3aVFZMlNsU29mcDRmTDU0bWRhTHZDczd1TDVtck1YY2VpJTJCUGxBUGFRS3V0UXIzUkZJS3RTZ1BIcmZJJTNE; cto_bidid=bktKe19SblJYdjl2VDlOVTdodlkwTlJUZDJzWmxpNFg1bmFxMVAlMkZ1dHBNWjJHaVlXcEx4a2J4VUpMWmhISlBBdVB6NjlzWUpkUW5BcklTMTdMeXE0eGxDZWclMkI2bEU2JTJCYVVUSlhkYnBGUkU5NFF3ayUzRA; __gads=ID=0ce9656e03877352:T=1700612445:RT=1700613242:S=ALNI_MbFig5r-aCy4HkFvS0jLB7bwPZDkQ; __gpi=UID=00000c923f44252a:T=1700612445:RT=1700613242:S=ALNI_MadX2SsbZQQnbKIHNrkvSNDD4t6Tg; STTS=18a7b79c1e20611c819547c25511b350%7C1700616104; _ga_FT1EL39NZ8=GS1.1.1700616104.2.0.1700616104.0.0.0',
    'Origin': 'https://fow.kr',
    'Referer': 'https://fow.kr/ranking',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'text/html, */*; q=0.01'
}


In [3]:
df = pd.DataFrame()
dic = {}

In [4]:
for g in range(200):
    payload = 'start='+str(g*50+1)
    response = requests.request("POST", url, headers=headers, data=payload)
    soup = BeautifulSoup(response.content,"html.parser")
    t=soup.find_all('td')
    for i in range(50):
        j = i*6
        dic['rank'] = int(str(t[j].get_text().split('위')[0])) #순위
        dic['name'] = t[j+1].get_text() #이름
        dic['more_url'] = t[j+1].find('a').get('href') #하이퍼링크
        dic['tier'] = t[j+2].get_text() #티어
        dic['LP'] = int(t[j+3].get_text()[:-1]) #LP
        dic['wins'] = int(t[j+4].get_text().split('승')[0])#승
        dic['loses'] = int(t[j+4].get_text().split('승')[1].split('패')[0])#패
        for k in range(5): #모스트 챔프 top 5
            try:
                dic['champion'+str(k)] = t[j+5].find_all('img')[k].get('tipsy').split('<BR>')[0] #챔프
                dic['champion_matches'+str(k)] =int(t[j+5].find_all('img')[k].get('tipsy').split('<BR>')[2].split('전')[0]) #전
                dic['champion_wins'+str(k)] =int(t[j+5].find_all('img')[k].get('tipsy').split('<BR>')[2].split('전')[1].split('승')[0]) #승
                dic['champion_loses'+str(k)] = int(t[j+5].find_all('img')[k].get('tipsy').split('<BR>')[2].split('전')[1].split('승')[1].split('패')[0]) #패
            except:
                dic['champion'+str(k)]=dic['champion_matches'+str(k)]=dic['champion_wins'+str(k)]=dic['champion_loses'+str(k)]=np.nan
        df_dic = pd.DataFrame(dic,index=[0])
        df = pd.concat([df, df_dic], ignore_index=True)

In [5]:
df.to_csv("/Users/jongmin/Downloads/lol_top_10000.csv", mode='w')